In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

   
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import json
import requests 

import pandas as pd
import requests
from bs4 import BeautifulSoup

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

# All requested packages already installed.



In [2]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [3]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
#print(df[0].to_json(orient='records'))

In [4]:
data = pd.DataFrame(df[0])


In [5]:
data_processed = data[data['Borough']!='Not assigned']


In [6]:
data_processed=data_processed.groupby('Postcode').agg(lambda x:','.join(set(x)))


In [7]:
data_processed.loc[data_processed['Neighbourhood']=='Not assigned','Neighbourhood'] = data_processed.loc[data_processed['Neighbourhood']=='Not assigned','Borough']


In [8]:
data_processed.shape


(103, 2)

### Final  output for the 1st part of assignment


In [9]:
data_processed


,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
M1E,Scarborough,"Morningside,Guildwood,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Kennedy Park,Ionview,East Birchmount Park"
M1L,Scarborough,"Golden Mile,Oakridge,Clairlea"
M1M,Scarborough,"Scarborough Village West,Cliffside,Cliffcrest"


### Second Part of the Assigment

In [10]:
!wget -q -O 'newyork_data.csv' http://cocl.us/Geospatial_data


In [11]:
post_code = pd.read_csv('newyork_data.csv')


In [12]:
data_processed = data_processed.merge(post_code,left_on='Postcode',right_on='Postal Code')


In [13]:
data_processed.head()


,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,Scarborough,"Malvern,Rouge",M1B,43.806686,-79.194353
1,Scarborough,"Port Union,Rouge Hill,Highland Creek",M1C,43.784535,-79.160497
2,Scarborough,"Morningside,Guildwood,West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


### Third Part of the assignment

In [14]:
CLIENT_ID = 'VEC3ZDLCCA1DS2DXDTG45MHFCPN4ZKWIHUEAP25AK3CB0F4G'
CLIENT_SECRET = 'UMKXIMLLVWJIGKQTNK3SUUBWTBPHQZUQHZMQWSJL1XYK05DB'
VERSION = '20180605'
LIMIT = 30

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [16]:
toronto_venues = getNearbyVenues(names=data_processed['Neighbourhood'],
                                   latitudes=data_processed['Latitude'],
                                   longitudes=data_processed['Longitude']
                                  )

Malvern,Rouge
Port Union,Rouge Hill,Highland Creek
Morningside,Guildwood,West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park,Ionview,East Birchmount Park
Golden Mile,Oakridge,Clairlea
Scarborough Village West,Cliffside,Cliffcrest
Cliffside West,Birch Cliff
Dorset Park,Wexford Heights,Scarborough Town Centre
Maryvale,Wexford
Agincourt
Tam O'Shanter,Sullivan,Clarks Corners
Milliken,Agincourt North,L'Amoreaux East,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Oriole,Henry Farm
Bayview Village
York Mills,Silver Hills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South,Flemingdon Park
Downsview North,Bathurst Manor,Wilson Heights
Northwood Park,York University
Downsview East,CFB Toronto
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
India Bazaar,The Beac

In [17]:
print(toronto_venues.shape)
toronto_venues.head()


(1338, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern,Rouge",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern,Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
3,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum


In [18]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))


There are 238 uniques categories.


In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Malvern,Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Malvern,Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Port Union,Rouge Hill,Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Port Union,Rouge Hill,Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Port Union,Rouge Hill,Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring the surrounding areas

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,"Beaumond Heights,Humbergate,Mount Olive,Jamest...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,"Bedford Park,Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.045455,0.000000
4,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.033333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
5,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.055556
6,"Cabbagetown,St. James Town",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
7,Caledonia-Fairbanks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.200000,0.000000
8,Canada Post Gateway Processing Centre,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
9,Cedarbrae,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [21]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
            venue  freq
0  Breakfast Spot  0.25
1  Clothing Store  0.25
2          Lounge  0.25
3    Skating Rink  0.25
4    Neighborhood  0.00


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2  Japanese Restaurant  0.25
3                 Café  0.25
4    Accessories Store  0.00


----Beaumond Heights,Humbergate,Mount Olive,Jamestown,Albion Gardens,South Steeles,Silverstone,Thistletown----
                  venue  freq
0           Pizza Place   0.2
1         Grocery Store   0.2
2  Fast Food Restaurant   0.1
3              Pharmacy   0.1
4           Coffee Shop   0.1


----Bedford Park,Lawrence Manor East----
                  venue  freq
0           Coffee Shop  0.09
1    Italian Restaurant  0.09
2  Fast Food Restaurant  0.05
3           Pizza Place  0.05
4      Sushi Restaurant  0.05


----Berczy Park----
            venue  freq
0  Farmers Market  0.07
1            Café  0.07
2        Beer Bar  0.07
3   

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Lounge,Clothing Store,Skating Rink,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
1,Bayview Village,Japanese Restaurant,Chinese Restaurant,Bank,Café,Yoga Studio,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
2,"Beaumond Heights,Humbergate,Mount Olive,Jamest...",Pizza Place,Grocery Store,Coffee Shop,Sandwich Place,Fast Food Restaurant,Beer Store,Pharmacy,Fried Chicken Joint,Empanada Restaurant,Electronics Store
3,"Bedford Park,Lawrence Manor East",Italian Restaurant,Coffee Shop,Pharmacy,Comfort Food Restaurant,Butcher,Restaurant,Café,Pub,Pizza Place,Liquor Store
4,Berczy Park,Cocktail Bar,Coffee Shop,Beer Bar,Farmers Market,Seafood Restaurant,Café,Basketball Stadium,Bistro,Breakfast Spot,Clothing Store


### Clustering the area into 5 clusters


In [24]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 2, 2, 2, 2, 1, 2, 0], dtype=int32)

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [26]:
toronto_merged = data_processed

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Malvern,Rouge",M1B,43.806686,-79.194353,2.0,Fast Food Restaurant,Print Shop,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,Scarborough,"Port Union,Rouge Hill,Highland Creek",M1C,43.784535,-79.160497,0.0,History Museum,Golf Course,Bar,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
2,Scarborough,"Morningside,Guildwood,West Hill",M1E,43.763573,-79.188711,0.0,Pizza Place,Rental Car Location,Medical Center,Mexican Restaurant,Electronics Store,Intersection,Breakfast Spot,Cosmetics Shop,Coworking Space,Comfort Food Restaurant
3,Scarborough,Woburn,M1G,43.770992,-79.216917,2.0,Coffee Shop,Korean Restaurant,Insurance Office,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476,0.0,Hakka Restaurant,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Creperie,Cuban Restaurant,Falafel Restaurant


In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Port Union,Rouge Hill,Highland Creek",0.0,History Museum,Golf Course,Bar,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
2,"Morningside,Guildwood,West Hill",0.0,Pizza Place,Rental Car Location,Medical Center,Mexican Restaurant,Electronics Store,Intersection,Breakfast Spot,Cosmetics Shop,Coworking Space,Comfort Food Restaurant
4,Cedarbrae,0.0,Hakka Restaurant,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Creperie,Cuban Restaurant,Falafel Restaurant
5,Scarborough Village,0.0,Playground,Convenience Store,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
6,"Kennedy Park,Ionview,East Birchmount Park",0.0,Discount Store,Coffee Shop,Department Store,Convenience Store,Chinese Restaurant,Electronics Store,Dog Run,Drugstore,Dumpling Restaurant,Eastern European Restaurant
10,"Dorset Park,Wexford Heights,Scarborough Town C...",0.0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Light Rail Station,Latin American Restaurant,Chinese Restaurant,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run
11,"Maryvale,Wexford",0.0,Middle Eastern Restaurant,Breakfast Spot,Sandwich Place,Bakery,Auto Garage,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
12,Agincourt,0.0,Breakfast Spot,Lounge,Clothing Store,Skating Rink,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
13,"Tam O'Shanter,Sullivan,Clarks Corners",0.0,Pizza Place,Thai Restaurant,Noodle House,Chinese Restaurant,Shopping Mall,Breakfast Spot,Fast Food Restaurant,Bank,Italian Restaurant,Fried Chicken Joint
15,L'Amoreaux West,0.0,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Sandwich Place,Camera Store,Breakfast Spot,Nail Salon,Pizza Place,Grocery Store


In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Milliken,Agincourt North,L'Amoreaux East,Steel...",1.0,Playground,Coffee Shop,Park,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
23,York Mills West,1.0,Convenience Store,Flower Shop,Park,Bank,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
25,Parkwoods,1.0,Park,Food & Drink Shop,Fast Food Restaurant,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
30,"Downsview East,CFB Toronto",1.0,Airport,Park,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
40,East Toronto,1.0,Park,Pizza Place,Convenience Store,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
44,Lawrence Park,1.0,Park,Lake,Swim School,Bus Line,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
48,"Moore Park,Summerhill East",1.0,Playground,Park,Restaurant,Department Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
50,Rosedale,1.0,Park,Playground,Building,Trail,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
64,"Forest Hill West,Forest Hill North",1.0,Jewelry Store,Park,Sushi Restaurant,Trail,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Drugstore,Eastern European Restaurant
72,Glencairn,1.0,Japanese Restaurant,Park,Pub,Bakery,Yoga Studio,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant


In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern,Rouge",2.0,Fast Food Restaurant,Print Shop,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
3,Woburn,2.0,Coffee Shop,Korean Restaurant,Insurance Office,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
7,"Golden Mile,Oakridge,Clairlea",2.0,Bus Line,Bakery,Fast Food Restaurant,Intersection,Bus Station,Soccer Field,Park,Cosmetics Shop,Convenience Store,Falafel Restaurant
8,"Scarborough Village West,Cliffside,Cliffcrest",2.0,American Restaurant,Motel,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
9,"Cliffside West,Birch Cliff",2.0,Café,General Entertainment,Skating Rink,College Stadium,Construction & Landscaping,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
17,Hillcrest Village,2.0,Fast Food Restaurant,Mediterranean Restaurant,Pool,Golf Course,Dog Run,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
18,"Fairview,Oriole,Henry Farm",2.0,Clothing Store,Coffee Shop,Food Court,Japanese Restaurant,Fast Food Restaurant,Department Store,Electronics Store,Juice Bar,Candy Store,Smoothie Shop
19,Bayview Village,2.0,Japanese Restaurant,Chinese Restaurant,Bank,Café,Yoga Studio,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
20,"York Mills,Silver Hills",2.0,Cafeteria,Yoga Studio,College Stadium,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
22,Willowdale South,2.0,Ramen Restaurant,Coffee Shop,Sandwich Place,Café,Restaurant,Indonesian Restaurant,Steakhouse,Japanese Restaurant,Fast Food Restaurant,Juice Bar


In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,"Newtonbrook,Willowdale",3.0,Piano Bar,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
97,"Humberlea,Emery",4.0,Baseball Field,Yoga Studio,Field,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
